In [1]:
# Developer docs
# https://www.tensorflow.org/tutorials/keras/classification
# https://www.tensorflow.org/api_docs/python/tf/keras/Model

# Importing modules
from os import listdir, system
import pandas as pd
import numpy as np
import sys
import time
import pandas as pd

import cv2 as cv2
import imutils
import imageio
import pickle
from IPython.display import clear_output, display, HTML


from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import tensorflow as tf
from skimage.io import imread
from skimage import measure
from skimage.measure import regionprops
from skimage.transform import resize
from scipy import ndimage


# Downloading dataset from: http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/
# You can comment this after running once.

### CHARS74K DATASET FOR TRAINING
# !wget http://www.ee.surrey.ac.uk/CVSSP/demos/chars74k/EnglishImg.tgz
# !tar -xvzf ./EnglishImg.tgz && rm -rf EnglishImg.tgz
# !git clone https://github.com/captcha-breakers/D-training-segmentation.git

# CAPTCHA DATA: DOWNLOAD ANYONE & APPROPRIATELY CONFIGURE THE PATH
# !git clone https://github.com/captcha-breakers/dataset-simple-uppercase.git
# !git clone https://github.com/rdmpage/solving-captchas-code-examples.git
# !git clone https://github.com/captcha-breakers/dataset-pycaptcha-uppercase.git
# !git clone https://github.com/captcha-breakers/D-training-color-noise-trans.git
# !git clone https://github.com/captcha-breakers/D-captchas-simple-salt-n-pepper.git

In [2]:
# import keras
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
# from keras.layers import Conv2D, MaxPooling2D, Input
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Reshape
from tensorflow.keras.models import Model

In [3]:
# Setting up folders
base_dir = "./D-training-color-noise-trans/data/"
folders = listdir(base_dir)
folders.sort()
folders = folders[10:36]
print(folders)

# Setting image_size
image_size = (32, 32)

['Sample011', 'Sample012', 'Sample013', 'Sample014', 'Sample015', 'Sample016', 'Sample017', 'Sample018', 'Sample019', 'Sample020', 'Sample021', 'Sample022', 'Sample023', 'Sample024', 'Sample025', 'Sample026', 'Sample027', 'Sample028', 'Sample029', 'Sample030', 'Sample031', 'Sample032', 'Sample033', 'Sample034', 'Sample035', 'Sample036']


In [4]:
# Collecting images from folders
data = []
total_images = 0
for i in folders:
    all_images = listdir(base_dir+i)
    total_images+=len(all_images)
    currentlabel = int(i[6:])-11

    for name in all_images:
        image = cv2.imread(base_dir+i+"/"+name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, image_size)

        # Filtering
        image = cv2.GaussianBlur(image,(5,5),0)
        image = cv2.Laplacian(image,cv2.CV_64F)
        image = cv2.GaussianBlur(image,(5,5),0)
        kernel = np.ones((3, 3), np.uint8)
        image = cv2.filter2D(image,-1,kernel)
        # image = cv2.medianBlur(image,5)
        # image = cv2.dilate(image,kernel,iterations = 1)
        # image = cv2.erode(image,kernel,iterations = 1)
        # image = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
        # image = cv2.filter2D(image,-1,kernel)
        # image = cv2.blur(image,(5,5))
        image = image.reshape(32, 32, 1)


        data.append((image/255, currentlabel))
print("Total images: ", total_images)

Total images:  26000


In [5]:
# # Visualizing images
# imshow(data[6][0])#, plt.show()
# imshow(data[5][0]), plt.show()
# imshow(data[7][0]), plt.show()

In [6]:
# Shuffling data & making train/test
np.random.shuffle(data)
n = len(data)
p = int(80*n/100)

train_images = []
train_labels = []
test_images = []
test_labels = []
for i in range(n):
    if i<p:
        train_images.append(data[i][0])
        train_labels.append(data[i][1])
    else:
        test_images.append(data[i][0])
        test_labels.append(data[i][1])
# train_images = train_images.reshape(-1, 28, 28, 1)
# print(train_images)
#print(train_labels)
print(type(train_images))
# image_size =  image_size + (1,)
# # image_size.append(3) 
# # s = set(image_size)


print(image_size)

<class 'list'>
(32, 32)


In [7]:
# train_images = train_images.reshape(list(train_images.shape) + [1])
print(np.array(train_labels).shape)

(20800,)


In [8]:
# del model

In [9]:
# Making the model and training it
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32,32,1)),
    #tf.keras.Input(shape=(32, 32, 1))
    tf.keras.layers.Conv2D(
        filters=1, 
        kernel_size=(5, 5), 
        padding='same', 
        activation='relu', 
        input_shape = (32,32,1)
    ),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='valid'),
    tf.keras.layers.Conv2D(1, (1,1), strides=1, activation="relu"),
    tf.keras.layers.UpSampling2D((2, 2), interpolation="bilinear"),
    tf.keras.layers.Conv2D(1, (1,1), strides=1, activation="relu"),
    tf.keras.layers.Flatten(input_shape=(32,32,1)),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(26, activation='relu')
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()
model.fit(
    np.array(train_images), np.array(train_labels), epochs=5, batch_size = 500) 
np.array(train_labels)
# model.fit(np.array(train_images), np.array(train_labels),batch_size=100, epochs=3, verbose=1, validation_data=(x_test, s_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 1)         26        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 1)         2         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 1)         2         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

array([ 6,  9,  6, ..., 25, 22, 17])

In [10]:
# model = Sequential()

# model.add(Conv2D(32, (5, 5), strides = (1, 1), name = 'conv0', input_shape = (32, 32, 1)))

# model.add(BatchNormalization(axis = 3, name = 'bn0'))
# model.add(Activation('relu'))

# model.add(MaxPooling2D((2, 2), name='max_pool'))
# model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
# model.add(Activation('relu'))
# # model.add(AveragePooling2D((3, 3), name='avg_pool'))

# # model.add(GlobalAveragePooling2D())
# model.add(Dense(300, activation="relu", name='rl'))
# # model.add(Dropout(0.5)

In [11]:

# model.compile(
#     optimizer='adam',
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )
# # model.fit(np.array(train_images), np.array(train_labels), epochs=10)
# # model.fit(np.array(train_images), np.array(train_labels),batch_size=100, epochs=3, verbose=1, validation_data=(x_test, s_test))
# batch_size = 32
# nb_epoch = 500
# # %%time
# # Train model
# history = model.fit(np.array(train_images), np.array(train_labels),batch_size=batch_size,epochs=nb_epoch, validation_split=0.2,shuffle=True,verbose=2)

In [12]:
# input_layer1 = Input((25, 67, 1))
# print(input_layer1)

In [13]:
# input_layer = image_size
# # print(input_layer)
# x = Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(input_layer)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Conv2D(filters=48, kernel_size=(5, 5), padding='same', activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)

# x = Dropout(0.3)(x)
# x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.3)(x)

# out = [Dense(10, name='digit%d' % i, activation='softmax')(x) for i in range(NUM_OF_LETTERS)]
# model = Model(inputs=input_layer, outputs=out)

In [14]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(36, 3, activation='relu',
#                            kernel_regularizer=tf.keras.regularizers.l2(0.02),
#                            input_shape=(28, 28, 1)),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(10)
# ])

# # Model is the full model w/o custom layers
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

# model.fit(np.array(train_images), epochs=3)
# loss, acc = model.evaluate(test_data)
# # , np.array(train_labels)
# print("Loss {}, Accuracy {}".format(loss, acc))

In [15]:
# def conv_block(inputs, filters, pool=True):
#     x = Conv2D(filters, 3, padding="same")(inputs)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     x = Conv2D(filters, 3, padding="same")(x)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     if pool == True:
#         p = MaxPool2D((2, 2))(x)
#         return x, p
#     else:
#         return x

# # def build_unet(shape, num_classes):
# #     inputs = Input(shape)

# #     """ Encoder """
# #     # x1, p1 = conv_block(inputs, 16, pool=True)
# #     # x2, p2 = conv_block(p1, 32, pool=True)
# #     # x3, p3 = conv_block(p2, 48, pool=True)
# #     # x4, p4 = conv_block(p3, 64, pool=True)
# #     x1, p1 = conv_block(inputs, 1, pool=True)
# #     x2, p2 = conv_block(p1, 5, pool=True)
# #     x3, p3 = conv_block(p2, 10, pool=True)
# #     x4, p4 = conv_block(p3, 15, pool=True)

# #     """ Bridge """
# #     # b1 = conv_block(p4, 128, pool=False)
# #     b1 = conv_block(p4, 18, pool=False)

# #     """ Decoder """
# #     u1 = UpSampling2D((2, 2), interpolation="bilinear")(b1)
# #     c1 = Concatenate()([u1, x4])
# #     x5 = conv_block(c1, 15, pool=False)

# #     u2 = UpSampling2D((2, 2), interpolation="bilinear")(x5)
# #     c2 = Concatenate()([u2, x3])
# #     x6 = conv_block(c2, 10, pool=False)

# #     u3 = UpSampling2D((2, 2), interpolation="bilinear")(x6)
# #     c3 = Concatenate()([u3, x2])
# #     x7 = conv_block(c3, 5, pool=False)

# #     u4 = UpSampling2D((2, 2), interpolation="bilinear")(x7)
# #     c4 = Concatenate()([u4, x1])
# #     x8 = conv_block(c4, 1, pool=False)

#     """ Output layer """
#     output = Conv2D(num_classes, 1, padding="same", activation="softmax")(x8)

#     return Model(inputs, output)

# # # Model is the full model w/o custom layers
# # model.compile(optimizer='adam',
# #               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
# #               metrics=['accuracy'])

# # model.fit(np.array(train_images), epochs=3)
# # loss, acc = model.evaluate(test_data)
# # # , np.array(train_labels)
# # print("Loss {}, Accuracy {}".format(loss, acc))
# # model = build_unet((20, 20, 3), 26)
# # model.summary()

In [16]:
# system("mkdir -p models")
# model.save('./models/my_model_84.h5')

# # Predicting on test images
# probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
# predictions = probability_model.predict(np.array(test_images))

# p=0
# for i in range(len(test_images)):
#   if np.argmax(predictions[i]) == test_labels[i]:
#     p+=1

# print("Test accuracy: ", 100*p/len(test_images))

In [17]:
# !git clone https://github.com/captcha-breakers/ubiquitous-pancake.git

In [18]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
fil_dir = "./D-captchas-simple-salt-n-pepper/data/"
total = 0
char_p = 0
char_n = 0
cap_p = 0
cap_n = 0

for fil in listdir(fil_dir)[:1000]:
  image = cv2.imread(fil_dir + fil)
  # imshow(image), plt.show()
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  _, thresh = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)

  cnts, new = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] 
  captcha = np.invert(thresh)
  labelled_captcha = measure.label(captcha)

  character_dimensions = (
      0.25*captcha.shape[0], 
      0.95*captcha.shape[0], 
      0.05*captcha.shape[1], 
      0.3*captcha.shape[1]
  )
  min_height, max_height, min_width, max_width = character_dimensions

  characters = []
  charactersx = []
  counter=0
  column_list = []
  row_list = []
  d = []
  for regions in regionprops(labelled_captcha):
      y0, x0, y1, x1 = regions.bbox
      region_height = y1 - y0
      region_width = x1 - x0

      if region_height > min_height and region_height < max_height and region_width > min_width and region_width < max_width:
          roi = captcha[y0:y1, x0:x1]
          resized_char = cv2.copyMakeBorder(roi, 10, 10, 10, 10, cv2.BORDER_CONSTANT)
          resized_char = resize(roi, image_size)
          column_list.append(x0)
          d.append([x0, resized_char])
  d = sorted(d)
  predicted_captcha = ""  
  ind=0
  result = []
  probability_model = tf.keras.Sequential([model, tf.keras.layers.ReLU()])
  #probability_model = tf.keras.Model(new_model, tf.keras.layers.ReLU())
  #probability_model.summary()
  for _,each_character in d:
      x_p,y_p = 3,3
      each_character = cv2.copyMakeBorder(each_character, x_p, x_p, y_p, y_p, cv2.BORDER_CONSTANT)
      each_character = cv2.resize(each_character,image_size)

      # imshow(each_character), plt.show()

      each_character = np.expand_dims(each_character, axis=0)
      each_character = np.expand_dims(each_character, axis=-1)
     
      result, = probability_model.predict([each_character])
      
      r = np.argmax(result)
      # print(r, len(result))

      if fil[ind] == chr(65+r):char_p += 1
      else:char_n += 1
      
      predicted_captcha+=chr(65+r)

      ind += 1
  
  if predicted_captcha == fil[:6]: cap_p+=1
  else:cap_n+=1

  # Statistics
  stats = pd.DataFrame(
    [
        [char_p, char_n, 100*char_p/(char_p+char_n)], 
        [cap_p, cap_n, 100*cap_p/(cap_p+cap_n)]
    ], 
    index=["Char", "Captcha"],
    columns=["Pos", "Neg", "Acc"]
  )
  total+=1
  clear_output(wait=True)
  display(HTML(stats.to_html()))
  print("Current file: ", total)
  print(fil[:6], ":", predicted_captcha)

,Pos,Neg,Acc
Char,1250,200,86.206897
Captcha,154,100,60.629921
